In [1]:
# Imports
import requests  # send request
from bs4 import BeautifulSoup  # parse web pages
import pandas as pd  # csv
from time import sleep  # wait
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import re

In [2]:
# Define a request header (to prevent anti-scraping)
headers = {
    'authority': 'curlconverter.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'if-modified-since': 'Fri, 15 Jul 2022 21:44:42 GMT',
    'if-none-match': 'W/"62d1dfca-3a58"',
    'referer': 'https://curlconverter.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

 ### Title, Subjects, Authors, and abstract

In [59]:
url = "https://arxiv.org/list/cs/pastweek?skip=0&show=100"
response = requests.get(url, headers = headers)
print(response.status_code)
page = response.text
soup = BeautifulSoup(page, "html.parser")
# print(page)
indexs = soup.find_all("span", attrs={"class" : "list-identifier"})
titles = soup.findAll("div", attrs={"class" : "list-title mathjax"})
subjects = soup.findAll("div", attrs={"class" : "list-subjects"})

#print(page)

200


### title

In [45]:
# print(titles[0].prettify())
# print("#########")
pattern = re.compile(r'</span>(.*?)</div>', re.DOTALL)  
title_list = []

for title in titles:
    match = pattern.search(title.prettify())
    if match:
        title_list.append(match.group(1).strip())

### index

In [66]:
# print(str(indexs[0]))
# print("#########")
pattern = re.compile(r'<span class="list-identifier"><a href="(.*?)" title="Abstract">arXiv:(.*?)</a>', re.DOTALL)  
index_list = []

for index in indexs:
    match = pattern.search(str(index))
    if match:
        index_list.append(match.group(1).strip())
# index_list

<span class="list-identifier"><a href="/abs/2403.00762" title="Abstract">arXiv:2403.00762</a> [<a href="/pdf/2403.00762" title="Download PDF">pdf</a>, <a href="/format/2403.00762" title="Other formats">other</a>]</span>
#########


### subject

In [81]:
# print(str(subjects[0]))
# print("#")
# print(str(subjects[1]))
pattern1 = re.compile(r'<span class="primary-subject">(.*?)</span>', re.DOTALL)  
pattern2 = re.compile(r'</span>; (.*?)</div>', re.DOTALL)  

subject_list = []

for subject in subjects:
    match1 = pattern1.search(str(subject))
    match2 = pattern2.search(str(subject))
    if match2:
        subject_list.append(match1.group(1).strip()+"; "+ match2.group(1).strip())
    else:
        subject_list.append(match1.group(1).strip())

# subject_list

<div class="list-subjects">
<span class="descriptor">Subjects:</span> <span class="primary-subject">Computer Vision and Pattern Recognition (cs.CV)</span>
</div>
#
<div class="list-subjects">
<span class="descriptor">Subjects:</span> <span class="primary-subject">Computation and Language (cs.CL)</span>; Artificial Intelligence (cs.AI); Machine Learning (cs.LG)

</div>


100

### abs & author

In [104]:
url = "https://arxiv.org"
pattern = re.compile(r'<meta content=(.*?) property="og:description"/>')
abs_list = []
author_list = []
i = 0
j = 0
for index in index_list:
    urli = f"{url}{index}"
    print(f'j = [j], url = {urli}')
    response = requests.get(urli, headers = headers)
    soup = BeautifulSoup(page_abs, "html.parser")
    
    abstract = soup.findAll("meta", attrs={"property" : "og:description"})
    abs_str = str(abstract)
    match = pattern.search(abs_str)
    if match:
        abs_list.append(match.group(1).strip())
        print(f'i = {i}')
    i += 1
    time.sleep(3)
print(abs_list[99])
    


j = [j], url = https://arxiv.org/abs/2403.00762
i = 0
j = [j], url = https://arxiv.org/abs/2403.00758
i = 1
j = [j], url = https://arxiv.org/abs/2403.00752
i = 2
j = [j], url = https://arxiv.org/abs/2403.00745
i = 3
j = [j], url = https://arxiv.org/abs/2403.00743


KeyboardInterrupt: 

In [126]:
response = requests.get("https://arxiv.org/abs/2403.00762", headers = headers)
page_abs = response.text
soup = BeautifulSoup(page_abs, "html.parser")
abstract = soup.findAll("meta", attrs={"property" : "og:description"})
title_abs = soup.findAll("h1", attrs={"class" : "title mathjax"})
authors_abs = soup.find_all("meta", attrs={"name" : "citation_author"})

# title
title_str = str(title_abs)
pattern = re.compile(r'</span>(.*?)</h1>')
title = pattern.search(title_str)

# abstract
abs_str = str(abstract)
pattern = re.compile(r'<meta content=(.*?) property="og:description"/>')
match = pattern.search(abs_str)
abstract0 = match.group(1).strip()

# authors
authors = '"'

for author in authors_abs:
    author_str = str(author)
    authors += author_str[15:-26]+"; "

authors = authors[:-2]+'"'

# subject



In [129]:
print(str(title_abs))
title_str = str(title_abs)
pattern = re.compile(r'</span>(.*?)</h1>')
title = pattern.search(title_str)
print(title.group(1))

[<h1 class="title mathjax"><span class="descriptor">Title:</span>Point Could Mamba: Point Cloud Learning via State Space Model</h1>]
Point Could Mamba: Point Cloud Learning via State Space Model


In [89]:
print(page_abs)

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">

<head>  <title>[2403.00762] Point Could Mamba: Point Cloud Learning via State Space Model</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="apple-touch-icon" sizes="180x180" href="/static/browse/0.3.4/images/icons/apple-touch-icon.png">
  <link rel="icon" type="image/png" sizes="32x32" href="/static/browse/0.3.4/images/icons/favicon-32x32.png">
  <link rel="icon" type="image/png" sizes="16x16" href="/static/browse/0.3.4/images/icons/favicon-16x16.png">
  <link rel="manifest" href="/static/browse/0.3.4/images/icons/site.webmanifest">
  <link rel="mask-icon" href="/static/browse/0.3.4/images/icons/safari-pinned-tab.svg" color="#5bbad5">
  <meta name="msapplication-TileColor" content="#da532c">
  <meta name="theme-color" 